### 02/08/2023

This is a simple testing script to test the indexed dataset

In [18]:
%load_ext autoreload
%autoreload 2

from datasets.generic_index_dataset import GenericIndexedModule
from datasets.dataset_utils import pad, tokenise_and_mask
from models.identity_module import IdentityModule
from pathlib import Path
import os, pytorch_lightning as pl

direct = Path("tempdata/SMILES_dataset")
print(os.listdir(direct / "train"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['SMILES', 'Chemical', 'HSQC', 'HYUN_FP', 'R2-6144FP', 'MW']


### Standard Training Task

In [16]:
features = ["HSQC", "HYUN_FP"]
feature_handlers = [pad, None]
gim = GenericIndexedModule(direct, features, feature_handlers, len_override = 5)
model = IdentityModule()

In [17]:
trainer = pl.Trainer(gpus=1, max_epochs=2)
trainer.fit(model, gim)

TypeError: Trainer.__init__() got an unexpected keyword argument 'gpus'

### Moonshot Task

In [19]:
# Vocabulary
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX, DEFAULT_MAX_SEQ_LEN

vocab_path = "tempdata/chemformer/bart_vocab.txt"
chem_token_start = 272
tokeniser = MolEncTokeniser.from_vocab_file(
  vocab_path, REGEX, chem_token_start
)

features = ["HSQC", "SMILES"]
feature_handlers = [pad, tokenise_and_mask]
gim = GenericIndexedModule(direct, features, feature_handlers, len_override = 5, molbart_tokeniser = tokeniser)
model = IdentityModule()

In [24]:
trainer = pl.Trainer(max_epochs=2)
trainer.fit(model, gim)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | v    | Linear | 6     
--------------------------------
6         Trainable params
0         Non-trainable params
6         Total params
0.000     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
------------------------------
Val step with batch_idx 0
Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s]                              
------------------------------
Train step with batch_idx 0
Size of batch parameter: 2
(tens: torch.Size([5, 21, 3]))
	raw_smiles:<class 'tuple'>:
    ('Br.C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12', 'Br.C=CC1CN2CCC1CC2C(O)c1ccnc2ccc(OC)cc12', 'Br.C=CCN1CCc2c(cc(O)c(O)c2Br)C(c2ccccc2)C1', 'Br.C=CCN1CCc2c(cc(O)c(O)c2Cl)C(c2ccccc2)C1', 'Br.C=CCN1CCc2cc(O)c(O)cc2C(c2ccccc2)C1')
	target:<class 'torch.Tensor'>:(device(type='cuda', index=0), torch.Size([5, 41]))
    tensor([[283, 299, 272, 280, 272, 272, 274, 272, 279, 277, 272, 272, 272, 274,
             272, 272, 277, 272, 275, 285, 281, 273, 274, 273, 273, 282, 273, 277,
             273, 273, 273, 275, 285, 272, 281, 273, 273, 274, 277,   3,   0],
            [283, 299, 272, 280, 272, 272, 274, 272, 279, 277, 272, 272, 272, 274,
             2

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it, v_num=16]
